<a href="https://colab.research.google.com/github/marta-brasola/FinancialMarketAnalytics/blob/main/Financial_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Financial Market Analytics
Marta Brasola, Luca Sammarini

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
from tabulate import tabulate


import warnings
warnings.simplefilter("ignore", UserWarning)

In [2]:
custom_params = {
    "axes.spines.right": False,
    "axes.spines.top": False,
    "axes.grid": True,         # Enable grid
    "grid.color": "gray",  # Set grid color to light gray
    "grid.linestyle": "--",     # Set grid line style to dashed
    "grid.linewidth": 0.5,      # Set grid line width
    "figure.facecolor": "#fbfbfb",
    "axes.facecolor": "#fbfbfb"
}

sns.set_theme(style="ticks", rc=custom_params)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/financial/data/Euro_preprocessed.csv', header=[0,1], index_col=[0])
df = df.replace(0, np.nan)

## Assumptions

- azioni frazionabili all'infinito
- no costi di commission (ma abbiamo fatto un'analisi di turnover)
- capitale infinito
- portafoglio equally wighted
- shift degli fundamental factors che sono ricavati da dati aziendali di bilancio

In [ ]:
def get_log_returns(data):
  """
  function to get log returns from a dataframe
  """
  prices = data['PX_LAST'].reset_index()
  prices = prices.rename_axis(None, axis=1)
  prices = prices.replace(0, np.NAN)
  prices = prices.fillna(np.nan)
  prices = prices.set_index('Date')
  returns = np.log(prices/prices.shift(1))
  returns = returns.iloc[1::]
  return returns

In [ ]:
returns = get_log_returns(df)

indicators = df.columns.get_level_values(0).unique()

benchmark_returns = returns.mean(axis=1)

## Checking missing data

When conducting financial analysis, it is crucial to examine the presence of missing values in a dataset, as they can significantly impact the accuracy and reliability of the analysis. Missing values can arise due to various reasons such as data collection errors, system issues, or intentional non-disclosure.

One important consideration in financial analysis is the potential for survivor bias. Survivor bias occurs when only the data of entities that have "survived" or persisted until the present time are included in the analysis, while the data of entities that did not survive or were excluded are omitted. In a financial context, survivor bias can distort the analysis by providing an overly optimistic view of performance, as it neglects the experiences of entities that may have faced adverse outcomes.

To address this concern, it is essential to carefully examine and handle missing values in a comprehensive manner. This may involve imputing missing data using appropriate techniques, considering the reasons for missingness, or conducting sensitivity analyses to assess the impact of missing values on the overall results. By acknowledging and addressing missing values, financial analysts can enhance the robustness and accuracy of their analyses, avoiding potential pitfalls associated with survivor bias.

#### missing data for each factor

To check which factors have most null values we computed the number of null values over the total possible values, which is 111 months times 797 total stocks. We than performed a filter action to exclude those factors that have more than 50% of missing data in the entire stock universe.

In [ ]:
tot_values = 111*797
null_ind = df.isnull().groupby(level=0, axis=1).sum().sum(axis=0).reset_index()
null_ind.columns = ['indicators', 'null_values']
null_ind['perc_null_values'] = (null_ind['null_values'] / tot_values) * 100
null_ind.sort_values(by='perc_null_values', ascending=False)
ind_drop = list(null_ind[null_ind['perc_null_values']>50]['indicators'])
print(f"indicators to drop:\n")
pprint(ind_drop)

In [ ]:
df.drop(columns=ind_drop, inplace=True)
prova = len(df.columns.get_level_values(0).unique())
print(f"number of indicators remaining: {prova}")

### missing data analysis over time

After performing the analysis of missing values over the entire dataframe we wanted to see how null values distribute over time

In [ ]:
indicators = df.columns.get_level_values(0).unique()

# create dataframe to plot the data
null = {}

for ind in indicators:
  null[ind] = (
    df.stack().reset_index().drop('tickers',axis=1)
    .groupby('Date')[ind]
    .apply(lambda x: x.isnull().sum())
  )

null = pd.DataFrame.from_dict(null)
null = round(null / 797, 2) * 100

labels = df.index[::3]

plt.figure(figsize=(15,6))

# Plot the data
plt.plot(null)

plt.xticks(rotation=90, ha='center', labels=labels, ticks=range(0, len(df.index), 3))
plt.xticks(rotation=45, ha='right', labels=['']*len(df.index), ticks=range(len(df.index)), minor=True)

threshold = 80
indicators_with_high_nulls = null.columns[null.mean() > threshold / 100]
for indicator in indicators_with_high_nulls:
    plt.plot([], label=indicator)  # Empty plot to create space for legend

plt.legend(title='Indicators with >{}% Nulls'.format(threshold), loc='upper left', bbox_to_anchor=(1, 1))

plt.title('Percentage of missing values for each indicator over time', fontsize=16)

plt.show()

At the beginning the are many null values for most of the factors that where given but at the end we decided not drop any more data to avoid introducing any more biases into our analysis

## Preprocessing
In this part we created a few dataframe with different structures that will be useful later on to perform our analysis.

In [ ]:
df2 = df.copy(deep=True)
df2.drop(df2.head(1).index, inplace=True)

r = pd.DataFrame(returns.values, index=returns.index,
                 columns=pd.MultiIndex.from_product([['RETURNS'],
                                                     returns.columns]))

df2 = pd.concat([df2, r], axis=1)
data_stack = df2.stack()
data_stack.head()

## Univariate Ranking

Univariate stock ranking is a method of evaluating and ranking individual stocks based on a single, specific criterion or factor. Instead of considering multiple factors simultaneously, univariate stock ranking focuses on a single variable to assess the relative strength or performance of different stocks.

Univariate stock ranking simplifies the analysis by focusing on a single factor, allowing investors or analysts to quickly identify and compare stocks based on that specific criterion. However, it's important to note that a comprehensive stock analysis often involves considering multiple factors to gain a more holistic view of a stock's potential. Many investors combine univariate ranking with multivariate approaches for a more nuanced evaluation of investment opportunities.


Look-ahead bias refers to a situation in data analysis or financial modeling where information that would not have been known or available at a certain point in time is improperly used to make predictions or decisions for that past period. This bias occurs when future data or information is inadvertently included in historical datasets, leading to an inaccurate representation of what information was actually available at the time.

In financial analysis, look-ahead bias can distort the evaluation of investment strategies, backtesting results, or performance metrics. It may lead to overestimating the effectiveness of a strategy because it mistakenly incorporates information that was not available at the time the decisions were made.

First, we compiled a list of factors derived from balance sheet data. In our historical dataset, we applied lagging to account for reporting and release delays. A common strategy to address look-ahead bias involves understanding the delivery lag associated with the factors being utilized and incorporating an appropriate lag. For instance, if a factor is related to March but is only reported in April, it is advisable to use the t − 1 (previous time period) value of that factor in any historical estimations. This concept is elaborated upon in greater detail in the following discussion.


In [ ]:
financial_ratios = [
    "PE_RATIO",
    "FIVE_YR_AVG_PRICE_EARNINGS",
    "T12M_DIL_PE_CONT_OPS",
    "10_YEAR_MOVING_AVERAGE_PE",
    "PX_TO_TANG_BV_PER_SH",
    "CURRENT_EV_TO_12M_SALES",
    "CURRENT_EV_TO_T12M_EBITDA",
    "FIVE_YEAR_AVG_EV_TO_T12_EBITDA",
    "T12M_DIL_EPS_CONT_OPS",
    "TRAIL_12M_EBITDA_PER_SHARE",
    "TRAIL_12M_SALES_PER_SH",
    "NET_DEBT_PER_SHARE",
    "TANG_BOOK_VAL_PER_SH",
    "NORMALIZED_ACCRUALS_CF_METHOD",
    "EBITDA_MARGIN",
    "EBITDA_MARGIN_3YR_AVG",
    "CAP_EXPEND_TO_SALES",
    "T12M_DVD_PAYOUT_RATIO",
    "EQY_DPS_NET_5YR_GROWTH",
    "NORMALIZED_ROE",
    "5YR_AVG_RETURN_ON_EQUITY",
    "NORMALIZED_ACCRUALS_BS_METHOD",
    "PX_TO_BOOK_RATIO"
]

The provided Python functions offer a valuable toolset for quantitative financial analysis. The `calculate_quantile_returns` function takes a DataFrame containing stock-specific factor values, divides them into quantiles, and computes returns for each quantile. This facilitates the assessment of how a specified factor influences stock returns. Additionally, the `calculate_IR` function calculates the Information Ratio (IR), a metric that measures the excess return of a portfolio relative to a benchmark, considering the associated tracking error. These functions are useful for evaluating and comparing the performance of investment strategies based on specific factors, aiding in the identification of potential alpha-generating opportunities.

In [ ]:
def calculate_quantile_returns(data_frame, column_name, returns_df, num_quantiles=5):

    """
    Calculate quantile returns based on the specified factor in the DataFrame.
    """

    quantile_ranks = pd.DataFrame(index=data_frame.index, columns=data_frame.columns.levels[1])
    ind_rank = data_frame[column_name]
    ind_rank = ind_rank.replace(0, np.nan)

    # ranking stock for every date based on the values of the month
    # before (if we consider a fundamental factor)
    for date in ind_rank.index:

        if column_name in financial_ratios:
            row_values = ind_rank.shift(1).loc[date]
        else:
            row_values = ind_rank.loc[date]

        if row_values.count() > 1:
            ranks = pd.Series(row_values).rank(method='max')
            quintiles = pd.qcut(ranks, q=num_quantiles, labels=False)
            quantile_ranks.loc[date] = quintiles

    # calculate returns for each quantile
    quantile_dfs = {}
    portfolio_returns = pd.DataFrame()

    for quantile in range(num_quantiles):
        filtered_df = returns_df[quantile_ranks == quantile]
        filtered_df_shifted = filtered_df
        quantile_dfs[quantile] = filtered_df_shifted
        portfolio_returns[quantile] = quantile_dfs[quantile].mean(axis=1).dropna()

    return quantile_dfs, portfolio_returns


def calculate_IR(portfolio_returns, benchmark_returns):
    """
    Calculate the Information Ratio.
    """

    active_return = portfolio_returns-benchmark_returns
    tracking_error = np.std(portfolio_returns-benchmark_returns)


    return active_return.mean() / tracking_error # ci va o meno l'annualizzazione?

### Evaluate the performances

#### Information Ratio (IR) in Financial Analysis

The Information Ratio (IR) is a widely-used metric in financial analysis that assesses the risk-adjusted performance of an investment strategy or portfolio compared to a benchmark. It provides insight into the excess return generated per unit of tracking error, offering a measure of the portfolio manager's skill in generating returns beyond the benchmark.

**Calculation:**

The Information Ratio is calculated using the following formula:

$$\text{IR} = \frac{\text{Active Return}}{\text{Tracking Error}}$$

Where:
- **Active Return** is the portfolio return minus the benchmark return.
- **Tracking Error** is the standard deviation of the active return.

**Interpretation:**

- A positive Information Ratio indicates that the portfolio has outperformed the benchmark on a risk-adjusted basis.
- A higher IR suggests a better risk-adjusted performance.
- A negative IR implies underperformance relative to the benchmark.

#### Benchmark

As for the benchmark we decided to use a portfolio built with all the stock in the stock universe to test whether is worth it or not to select stock based our strategies

In [ ]:
# indicators = df.columns.get_level_values(0).unique()

indicator_IR = pd.DataFrame(index=indicators, columns=['IR_top_quantile', 'IR_bottom_quantile', 'tracking_top_quantile',
                                                       'tracking_error_bottom_quantile', 'returns_top_quantile', 'returns_bottom_quantile'])

for indicator in indicators:
    result_dfs, portfolio_returns = calculate_quantile_returns(df, indicator, returns)
    indicator_IR.at[indicator, 'IR_top_quantile'] = calculate_IR(portfolio_returns[4], benchmark_returns)
    indicator_IR.at[indicator, 'IR_bottom_quantile'] = calculate_IR(portfolio_returns[0], benchmark_returns)
    indicator_IR.at[indicator, 'tracking_top_quantile'] = np.std(portfolio_returns[4]-benchmark_returns)
    indicator_IR.at[indicator, 'tracking_error_bottom_quantile'] = np.std(portfolio_returns[0]-benchmark_returns)
    indicator_IR.at[indicator, 'returns_top_quantile'] = np.mean(portfolio_returns[4]) * 100
    indicator_IR.at[indicator, 'returns_bottom_quantile'] = np.mean(portfolio_returns[0]) * 100


indicator_IR.sort_values('IR_top_quantile', ascending = False)

In [ ]:
selected_rows = (
    (indicator_IR['IR_top_quantile'] > 0.50) |
    (indicator_IR['IR_bottom_quantile'] > 0.50)
)

indicator_IR[selected_rows]

- plot the returns of the selected rows over time

In [ ]:
# Selecting columns for plotting
selected_columns = ['IR_top_quantile', 'IR_bottom_quantile']

labels = ['CURRENT_EV_TO\nT12M_EBITDA', 'EQY_REC_CONS',
       'FIVE_YEAR_AVG\nEV_TO_T12_EBITDA', 'PE_RATIO', 'RSI_14D', 'RSI_30D',
       'RSI_9D']

# Plotting the selected data
ax = indicator_IR[selected_rows][selected_columns].plot(kind='bar', figsize=(10, 6))

Xstart, Xend = ax.get_xlim()
Ystart, Yend = ax.get_ylim()

ax.text(Xstart+0.5, Yend+0.2, '''IR Comparison Between Top and Bottom Quantiles of indicators with IR > 0.5''', fontsize=14)

plt.xlabel('indicators', fontsize=14)
plt.ylabel('Informatio Ratio', fontsize=14)
plt.legend(['First Quantile', 'Fifth Quantile'], fontsize=12)
ax.set_xticklabels(labels, rotation=35, ha='right')
plt.yticks(fontsize=12)
plt.show()



In [ ]:
# Selecting columns for plotting
selected_columns = ['returns_top_quantile', 'returns_bottom_quantile']

labels = ['CURRENT_EV_TO\nT12M_EBITDA', 'EQY_REC_CONS',
       'FIVE_YEAR_AVG\nEV_TO_T12_EBITDA', 'PE_RATIO', 'RSI_14D', 'RSI_30D',
       'RSI_9D']

# Plotting the selected data
ax = indicator_IR[selected_rows][selected_columns].plot(kind='bar', figsize=(10, 6))

Xstart, Xend = ax.get_xlim()
Ystart, Yend = ax.get_ylim()

ax.text(Xstart+0.5, Yend+0.2,'''Returns Comparison Between Top and Bottom Quantiles of indicators with IR > 0.5''', fontsize=14)

plt.xlabel('Indicators', fontsize=14)
plt.ylabel('Returns', fontsize=14)
plt.legend(['First Quantile', 'Fifth Quantile'], fontsize=12)
ax.set_xticklabels(labels, rotation=35, ha='right')
plt.yticks(fontsize=12)
plt.show()



In [ ]:
for ind in indicator_IR[selected_rows].index:
    result_dfs, portfolio_returns = calculate_quantile_returns(df, ind, returns)

    plt.figure(figsize=(15, 6))

    # Iterate through quantiles
    for quantile in range(len(portfolio_returns.columns)):
        plt.plot(portfolio_returns[quantile].cumsum(), label=f"Quantile {quantile}")

    plt.plot(benchmark_returns.cumsum(), 'k--', label='Benchmark Returns')
    plt.title(f"Cumulative Returns for {ind}")
    plt.xlabel("Date")
    plt.ylabel("Cumulative Returns")

    # Set x ticks for every 4 months
    plt.xticks(range(0, len(portfolio_returns), 4), portfolio_returns.index[::4], rotation=45)

    plt.legend()
    plt.show()


### Correlation

- correlazione e diversificazione

- a cosa serve l'analisi di correlation
- quali sono i metodi con cui si può effettuare un'analisi di correlation
- quale metdodo abbiamo scelto noi
- cos'è la correlation di kendall

In [ ]:
top = (
    (indicator_IR['IR_top_quantile'] > 0) |
    (indicator_IR['IR_bottom_quantile'] > 0)
)

len(indicator_IR[top].index)

In [ ]:
sns.set_palette('Set2')

In [ ]:
mov_avg = ['MOV_AVG_10D', 'MOV_AVG_20D', 'MOV_AVG_30D', 'MOV_AVG_40D', 'MOV_AVG_50D', 'MOV_AVG_5D']
cluster_mov_avg = df[indicator_IR[top].index][mov_avg].stack().corr(method='kendall')
fig, ax = plt.subplots(figsize=(8,6))
sns.heatmap(cluster_mov_avg, vmin=-1, vmax=1, cmap='Blues', annot=True, cbar=True)
Xstart, Xend = ax.get_xlim()
Ystart, Yend = ax.get_ylim()

ax.text(Xstart+0.5, Yend-0.2,

        '''Correlation between moving averages''', fontsize=20)

x_tick_positions = [0, 1, 2, 3, 4, 5]  # Adjust the positions as needed
#ax.set_xticks(x_§tick_positions)

ax.set_xticklabels(mov_avg, rotation=35, ha='right')
ax.set_yticklabels(mov_avg, rotation=0)

#ax.set_xticks([])
ax.xaxis.set_ticks_position('none')
for s in ['top','right','bottom','left']:
    ax.spines[s].set_visible(False)
#plt.savefig("new_heatmap.png")
#plt.tight_layout()
plt.show()

In [ ]:
rsi = ['RSI_9D', 'RSI_14D', 'RSI_30D']
cluster_rsi = df[indicator_IR[top].index][rsi].stack().corr(method='kendall')
fig, ax = plt.subplots(figsize=(7,5))
sns.heatmap(cluster_rsi, vmin=-1, vmax=1, cmap='Blues', annot=True, cbar=True)
Xstart, Xend = ax.get_xlim()
Ystart, Yend = ax.get_ylim()

ax.text(Xstart+0.4, Yend-0.2,

        '''Correlation between RSI''', fontsize=20)

x_tick_positions = [0, 1, 2]  # Adjust the positions as needed
#ax.set_xticks(x_§tick_positions)

ax.set_xticklabels(rsi, rotation=35, ha='right')
ax.set_yticklabels(rsi, rotation=0)

#ax.set_xticks([])
ax.xaxis.set_ticks_position('none')
for s in ['top','right','bottom','left']:
    ax.spines[s].set_visible(False)
#plt.savefig("new_heatmap.png")
#plt.tight_layout()
plt.show()

In [ ]:
top = [i for i in indicator_IR[indicator_IR['IR_top_quantile']>0.50].index]
top.extend(i for i in indicator_IR[indicator_IR['IR_bottom_quantile']>0.50].index)

In [ ]:
corr = df[top].stack().corr(method='kendall')
fig, ax = plt.subplots(figsize=(8,6))
sns.heatmap(corr, vmin=-1, vmax=1, cmap='Blues', annot=True, cbar=True)
Xstart, Xend = ax.get_xlim()
Ystart, Yend = ax.get_ylim()

ax.text(Xstart+0.2, Yend-0.2, '''Correlation between indicators with IR>0.50''', fontsize=14)

x_tick_positions = [0, 1, 2, 3, 4, 5,6]  # Adjust the positions as needed
#ax.set_xticks(x_tick_positions)

ax.set_xticklabels(labels, rotation=35, ha='right')
ax.set_yticklabels(labels, rotation=0)

#ax.set_xticks([])
ax.xaxis.set_ticks_position('none')
for s in ['top','right','bottom','left']:
    ax.spines[s].set_visible(False)
#plt.savefig("new_heatmap.png")
#plt.tight_layout()
plt.show()

### Turnover

We decided to set commission costs to zero, but still we decided to perform a Turnover analysis on the univariate strategies to see which indicators give a greater turnover over the stock selection that would lead to larger costs.


- analisi del turnover delle strategie univariate per tutti gli indicatori
- come lo abbiamo calcolato (assoluto, relativo, medio)
- commento dei risultati e grafici

In [ ]:
# saving in the dataframes 'first_quantile_stocks' and 'last_quantile_stocks'
# all the tickers of the stocks that have been selected for every timestamp

first_quantile_stocks = pd.DataFrame(index=df.index, columns=indicators)
last_quantile_stocks = pd.DataFrame(index=df.index, columns=indicators)


# Loop through each indicator
for ind in indicators:
    result_dfs, portfolio_returns = calculate_quantile_returns(df, ind, returns)

    # For each date, store the list of best stocks in the corresponding column
    for index, row in result_dfs[4].iterrows():
        first_quantile_stocks.loc[index, ind] = list(row.dropna().index)

    for index, row in result_dfs[0].iterrows():
        last_quantile_stocks.loc[index, ind] = list(row.dropna().index)


first_quantile_date = first_quantile_stocks.reset_index()
last_quantile_date = last_quantile_stocks.reset_index()


turnover_first_quantile = pd.DataFrame(index = df.index)
turnover_last_quantile = pd.DataFrame(index = df.index)

lengths_first_quantile = pd.DataFrame(index = df.index)
lengths_last_quantile = pd.DataFrame(index = df.index)


# this function calculates the turnover by confronting the stocks in the
# portfolio at the time t and t-1
def turnover_select(indicator, stocks, stocks_date):


    expanded_df2 = stocks[indicator].explode()
    merged_df = pd.merge(stocks_date[[indicator, 'Date']],
                         expanded_df2, how='right',
                         left_on='Date', right_on='Date')

    changed_stocks = (
        merged_df.groupby('Date')[indicator + '_y']
        .apply(set)
        .diff()
        .apply(lambda x: list(x) if isinstance(x, set) else [])
    )

    lenght_stocks = (
      merged_df.groupby('Date')[indicator+ '_y']
      .count()
    )

    if stocks.equals(first_quantile_stocks) and stocks_date.equals(first_quantile_date):
      lengths_first_quantile[indicator] = lenght_stocks
      turnover_first_quantile[indicator] = changed_stocks.apply(lambda x: len(x))

    elif stocks.equals(last_quantile_stocks) and stocks_date.equals(last_quantile_date):
      lengths_last_quantile[indicator] = lenght_stocks
      turnover_last_quantile[indicator] = changed_stocks.apply(lambda x: len(x))

# apply the function for every indicator in the dataset
for ind in indicators:

    turnover_select(indicator=ind,
                    stocks=first_quantile_stocks,
                    stocks_date=first_quantile_date)

    turnover_select(indicator=ind,
                    stocks=last_quantile_stocks,
                    stocks_date= last_quantile_date)


# we dropped the first two rows of the dataset since there are no stock in the portfolio
# in january of 2003 and also the second row becaues every stock is different and
# would later affect the calculations of the summary statistics
turnover_first_quantile.drop(['2003-01-31', '2003-02-28'], axis=0, inplace=True)
turnover_last_quantile.drop(['2003-01-31', '2003-02-28'], axis=0, inplace=True)
lengths_first_quantile.drop(['2003-01-31', '2003-02-28'], axis=0, inplace=True)
lengths_last_quantile.drop(['2003-01-31', '2003-02-28'], axis=0, inplace=True)

In [ ]:
labels = ['EQY_REC_CONS', 'RSI_14D', 'RSI_30D', 'RSI_9D',
          'CURRENT_EV_TO_T12M_EBITDA', 'FIVE_YEAR_AVG_EV_TO_T12_EBITDA', 'PE_RATIO']

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Plot and format the first subplot
for column, label in zip(top, labels):
    ax1.plot(turnover_first_quantile[column].rolling(6).mean(), label=label)
ax1.set_title('Turnover - First Quantile - Indicators with IR>0.5')
ax1.grid(True, which='both', linestyle='--', linewidth=0.5)  # Add gridlines
ax1.legend()  # Add legend

# Plot and format the second subplot
ax2.plot(turnover_first_quantile[top].rolling(6).mean())
ax2.set_title('Turnover - Last Quantile - Indicators with IR>0.5')
ax2.grid(True, which='both', linestyle='--', linewidth=0.5)  # Add gridlines

xticks_major = ax2.get_xticks()[1::6]  # Adjust the step as needed
xtick_labels = turnover_first_quantile.index[xticks_major]
ax2.set_xticks(xticks_major)
ax2.set_xticklabels(xtick_labels, rotation=45, ha='right')

plt.tight_layout()
plt.show()


As we can see the indicator that generates the most turnover is 'EQY_REC_CONS' followed by momentum indicators in order of their frequency. The indicator that has the least turnover, from the ones selected is 'FIVE_YEAR_AVG_EV_TO_T12_EBITDA'.

In [ ]:
# We are calculating some summary statistics of the turnver for each indicator

rel_first_quantile = round(turnover_first_quantile / lengths_first_quantile,2) * 100
rel_last_quantile = round(turnover_last_quantile / lengths_last_quantile,2) * 100

stats = round(turnover_first_quantile.mean(),2).reset_index()
stats.columns = ['indicators', 'average_first_quantile']

stats2 = round(turnover_last_quantile.mean(),2).reset_index()
stats2.columns = ['indicators', 'average_last_quantile']

stats3 = round(rel_first_quantile.mean(),2).reset_index()
stats3.columns = ['indicators', 'average_perc_first_quantile']

stats4 = round(rel_last_quantile.mean(),2).reset_index()
stats4.columns = ['indicators', 'average__perc_last_quantile']

stats3 = stats3.merge(stats4, how='inner', on='indicators')
stats = stats.merge(stats2, how='inner', on='indicators')

stats = stats.merge(stats3, how='inner', on='indicators')

In [ ]:
stats.sort_values(by='average_first_quantile', ascending=False)

We divided indicators into three main categories:
1. value measures
2. profitability measures
3. momentum based measures

This will give us more insight into the turnover behaviour but it will be useful later for the multivariate strategy

In [ ]:
indicator_categories = {
    '5YR_AVG_RETURN_ON_EQUITY': 'Profitability',
    'CURRENT_EV_TO_12M_SALES': 'Value',
    'CURRENT_EV_TO_T12M_EBITDA': 'Value',
    'CUR_MKT_CAP': 'Value',
    'EBITDA_MARGIN': 'Profitability',
    'EBITDA_MARGIN_3YR_AVG': 'Profitability',
    'EQY_REC_CONS': 'Momentum',
    'FIVE_YEAR_AVG_EV_TO_T12_EBITDA': 'Value',
    'FIVE_YR_AVG_PRICE_EARNINGS': 'Value',
    'MOV_AVG_10D': 'Momentum',
    'MOV_AVG_20D': 'Momentum',
    'MOV_AVG_30D': 'Momentum',
    'MOV_AVG_40D': 'Momentum',
    'MOV_AVG_50D': 'Momentum',
    'MOV_AVG_5D': 'Momentum',
    'NET_DEBT_PER_SHARE': 'Profitability',
    'NORMALIZED_ACCRUALS_BS_METHOD': 'Profitability',
    'NORMALIZED_ACCRUALS_CF_METHOD': 'Profitability',
    'NORMALIZED_ROE': 'Profitability',
    'OPERATING_ROIC': 'Profitability',
    'PE_RATIO': 'Value',
    'PX_LAST': np.nan,  # This one doesn't fall neatly into a single category
    'PX_TO_BOOK_RATIO': 'Value',
    'PX_TO_TANG_BV_PER_SH': 'Value',
    'RSI_14D': 'Momentum',
    'RSI_30D': 'Momentum',
    'RSI_9D': 'Momentum',
    'TANG_BOOK_VAL_PER_SH': 'Profitability',
    'TRAIL_12M_EBITDA_PER_SHARE': 'Value',
    'TRAIL_12M_SALES_PER_SH': 'Value',
    'VOLATILITY_180D': 'Profitability',
    'VOLATILITY_30D': 'Profitability',
    'VOLATILITY_90D': 'Profitability',
    'WACC_COST_EQUITY': 'Profitability'
}

stats['Category'] = stats['indicators'].map(indicator_categories)

In [ ]:
stats.groupby('Category')[['average_first_quantile','average_last_quantile','average_perc_first_quantile','average__perc_last_quantile']].mean()

Overall momentum and profitability measures lead to an higher turnover in the stock selection process rather than univariate Value strategies.

## Multivariate strategy  

Multivariate stock ranking with the Z-score, unlike the univariate one, involves the simultaneous consideration of multiple financial metrics to assess and rank the performance of stocks within a given universe. This methodology integrates statistical analysis, particularly the Z-score, to create a composite indicator that aids investors in making more informed decisions based on a holistic evaluation of a company's financial health.

In this approach, the Z-score, a statistical measure standardizing and quantifying a data point's deviation from the mean within a dataset, is applied to various financial ratios and indicators. This facilitates the comparison of stocks across different criteria.

Multivariate stock ranking extends beyond traditional univariate approaches by incorporating a diverse set of financial metrics. The amalgamation of these factors provides a more comprehensive assessment of a stock's overall performance.

First of all, we calculated the single Z-scores for every indicator, normalizing all the values of the dataset. After that, we calculated the composite Z-score for every stock, by summing the single Z-scores of the selected indicators, using the performance obtained in the univariate strategy to choose the sign of the influence of the particular indicator on the overall Z-score.
We used different methods to select the indicators to include. The first method considers all the indicators that performed best in the Univariate strategy: we selected all the indicators that had an IR greater than 0.5 in the top or bottom quantiles, using the negative values for computing the Z-score for the bottom quantiles. This is a procedure that only takes into account the univariate results, without considering the nature of the indicators selected. For this reason, many indicators representing similar features can be selected.

In [ ]:
def calculate_quantile_returns_zscore(data_frame, column_name, returns_df, num_quantiles=5):
    """
    Calculate quantile returns based on the specified column in the DataFrame.
    """

    quantile_ranks = pd.DataFrame(index=data_frame.index, columns=data_frame.columns)
    ind_rank = data_frame[column_name]
    ind_rank = ind_rank.replace(0, np.nan)

    for date in ind_rank.index:
        row_values = ind_rank.loc[date]
        if row_values.count() > 1:
          ranks = pd.Series(row_values).rank(method='max')
          quintiles = pd.qcut(ranks, q=num_quantiles, labels=False)
          quantile_ranks.loc[date] = quintiles

    quantile_dfs = {}
    portfolio_returns = pd.DataFrame()

    for quantile in range(num_quantiles):
        filtered_df = returns_df[quantile_ranks == quantile]
        filtered_df_shifted = filtered_df
        quantile_dfs[quantile] = filtered_df_shifted
        portfolio_returns[quantile] = quantile_dfs[quantile].mean(axis=1).dropna()

    return quantile_dfs, portfolio_returns

In [ ]:
z_score = (df-df.T.groupby(level=0).mean().T)/df.T.groupby(level=0).std().T

levels = [0.1, 0.25, 0.5, 1, 1.5, 2]
indicator_IR_z = pd.DataFrame(index=levels, columns=['IR_max_quant', 'tracking_error_max', 'returns_max'])

for selection_level in levels:
    sel_z_score = z_score[indicator_IR[(indicator_IR['IR_top_quantile']>selection_level)].index].merge(-z_score[indicator_IR[(indicator_IR['IR_bottom_quantile']>selection_level)].index], on='Date')
    final = sel_z_score.T.groupby(level=1).sum().T

    result_dfs_2, portfolio_returns_2 = calculate_quantile_returns_zscore(final, final.columns, returns)

    plt.figure(figsize=(15, 6))

    # Iterate through quantiles
    for quantile in range(len(portfolio_returns_2.columns)):
        plt.plot(portfolio_returns_2[quantile].cumsum(), label=f"Quantile {quantile}")

    plt.title(f"Cumulative Returns for IR > {selection_level}")
    plt.xlabel("Date")
    plt.ylabel("Cumulative Returns")

    # Set x ticks for every 4 months
    plt.xticks(range(0, len(portfolio_returns_2), 4), portfolio_returns_2.index[::4], rotation=45)

    plt.legend()
    plt.show()

    indicator_IR_z.at[selection_level, 'IR_max_quant'] = calculate_IR(portfolio_returns_2[4], benchmark_returns)
    indicator_IR_z.at[selection_level, 'tracking_error_max'] = np.std(portfolio_returns_2[4]-benchmark_returns)
    indicator_IR_z.at[selection_level, 'returns_max'] = np.mean(portfolio_returns_2[4]) * 100

The graphs show the returns changing the level of IR treshold used to select the factors to compute the composite Z-score.

In [ ]:
indicator_IR_z

This strategy, while mantaining the returns almost as good as the best univariate factors, is able to reduce tracking errors, thus mantaining excellent IRs.

As a second strategy, we computed a composite Z-score using factors that could describe different features of the stocks: we selected one momentum indicator, two value indicators and one profitability indicator, taking into account univariate performance and checking the correlation between the factors.

In [ ]:
indicator_IR_z_2 = pd.DataFrame(columns=['IR_max_quant', 'tracking_error_max', 'returns_max'])

pos_factors = [
    "RSI_9D",
]

neg_factors = [
    "PE_RATIO",
    "PX_TO_BOOK_RATIO",
    "NORMALIZED_ACCRUALS_CF_METHOD"
]

sel_z_score = z_score[pos_factors].merge(-z_score[neg_factors], on='Date')
final = sel_z_score.T.groupby(level=1).sum().T

corr = sel_z_score.stack().corr(method='kendall')
fig, ax = plt.subplots(figsize=(8,6))
sns.heatmap(corr, vmin=-1, vmax=1, cmap='Blues', annot=True, cbar=True)
Xstart, Xend = ax.get_xlim()
Ystart, Yend = ax.get_ylim()

ax.text(Xstart+1.4, Yend-0.2,

        '''Correlation between chosen
                indicators''', fontsize=20)

x_tick_positions = [0, 1, 2, 3, 4, 5,6]  # Adjust the positions as needed
#ax.set_xticks(x_tick_positions)

# ax.set_xticklabels(labels, rotation=35, ha='right')
# ax.set_yticklabels(labels, rotation=0)

#ax.set_xticks([])
ax.xaxis.set_ticks_position('none')
for s in ['top','right','bottom','left']:
    ax.spines[s].set_visible(False)
#plt.savefig("new_heatmap.png")
#plt.tight_layout()
plt.show()

In [ ]:
result_dfs_3, portfolio_returns_3 = calculate_quantile_returns_zscore(final, final.columns, returns)

plt.figure(figsize=(15, 6))

# Iterate through quantiles
for quantile in range(len(portfolio_returns_3.columns)):
    plt.plot(portfolio_returns_3[quantile].cumsum(), label=f"Quantile {quantile}")

plt.title(f"Cumulative Returns")
plt.xlabel("Date")
plt.ylabel("Cumulative Returns")

# Set x ticks for every 4 months
plt.xticks(range(0, len(portfolio_returns_3), 4), portfolio_returns_3.index[::4], rotation=45)

plt.legend()
plt.show()


indicator_IR_z_2.at[0, 'IR_max_quant'] = calculate_IR(portfolio_returns_3[4], benchmark_returns)
indicator_IR_z_2.at[0, 'tracking_error_max'] = np.std(portfolio_returns_3[4]-benchmark_returns)
indicator_IR_z_2.at[0, 'returns_max'] = np.mean(portfolio_returns_3[4]) * 100

In [ ]:
indicator_IR_z_2

We can see that the result of this method outperform the results of the multivariate method proposed before: the return obtained is slighty inferior but the Information Ratio and the Tracking Error show better values.

## Machine Learning to explain returns

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import make_pipeline
from sklearn import metrics

We implement a Random Forest Regressor that tries to explain the returns of the equity for the following month given the input factors used in the previous models. We compare the metrics of the regressor with a Linear Regression model to have a benchmark of the results obtained.

We choose to use a Random Forest because is well known to be one of the best performer models. Furthermore, it is very versatile and less computationally expensive than other machine learning models.

In [ ]:
subset = data_stack.loc['2003-03-31':'2004-02-27']

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = KNNImputer(n_neighbors=2)

subset_mean = imp_mean.fit_transform(subset)
subset_knn = imputer.fit_transform(subset)

subset = subset.values

In [ ]:
num_columns = subset.shape[1]
fig, axs = plt.subplots(num_columns, figsize=(10, 4*num_columns))

# Plot the distribution of each array in each subplot
for i in range(num_columns):
    axs[i].hist(subset[:, i], alpha=0.5, label='Subset', bins=20)
    axs[i].hist(subset_mean[:, i], alpha=0.5, label='Subset Mean', bins=20)
    axs[i].hist(subset_knn[:, i], alpha=0.5, label='Subset KNN', bins=20)
    axs[i].set_title(f'Column {i+1}')
    axs[i].legend()

plt.tight_layout()
plt.show()

### Random Forest

We implement a Random Forest Regressor through the Scikit Learn library. We use a rolling window to train and test the model, simulating a real time situation where the user knows the indicators and returns for the last months and tries to predict the returns for the following month given the indicators of the present month. The rolling window is 12 month wide, and is shifted by one month at every iteration. At each iteration a model is trained on the 12 month rolling window and tries to use the following month data to predict the returns. Then, the rolling window is shifted by one month and the process is repeated.

Finally, regression metrics are computing (MAE, MSE and RMSE), confronting the predictions with the actual returns of the stocks.

In [ ]:
def xs_forest(df, n_estimators=4):
    '''Build a cross-sectional forest model and
    output the pipeline at each timestamp'''
    # sklearn does not have an imputing method unless specified
    df = df[df['RETURNS'].notna()]
    # do not leave out test sample in our training pipeline
    X_train, y_train = df.shift(1).drop(columns='RETURNS').values, df['RETURNS'].values

    model = make_pipeline(KNNImputer(n_neighbors=3),
                          RandomForestRegressor(n_estimators=n_estimators,
                                                random_state=42))
    model.fit(X_train, y_train)
    # we return the model instead of slope estimations
    # such that we can transform the next-period data later
    return model

In [ ]:
ls_forest = {}
ts = df.index.to_list()
dt = 12                                # 1-year rolling window
for i in range(len(ts)-dt+1):
    df3 = data_stack.loc[ts[i]:ts[i+dt-1]]
    ls_forest[ts[i+dt-1]] = xs_forest(df3)    # add the model corresponding to i+dt-1, last period of rolling window
df_forest = pd.Series(ls_forest)

res = {}

for t in df_forest.shift(1).index.to_list():
    df4 = data_stack.loc[t]
    df4 = df4[df4['RETURNS'].notna()]
    X_test, y_test = df4.shift(1).drop(columns='RETURNS').values, df4['RETURNS'].values
    try:
        forest = df_forest.shift(1).loc[t]
        y_pred = forest.predict(X_test).reshape(-1)  # prediction array reshaped to 1d
    except:
        y_pred = np.nan
    res[t] = pd.DataFrame({'RETURNS': y_test, 'PREDICTIONS': y_pred}, index=df4.index)

res = pd.concat(res)
res.index.names = ['Date', 'ticker']

In [ ]:
feature_importance = pd.DataFrame()
for i in range(len(ts)-dt+1):
  feature_importance[ts[i+dt-1]] = ls_forest[ts[i+dt-1]][1].feature_importances_
impo_means = feature_importance.mean(axis = 1)
features = pd.Series(impo_means.values, data_stack.drop('RETURNS', axis=1).columns)
features.sort_values(ascending=False).plot.bar(fontsize=7, title='Feature Importance expressed as Gini impurity (average on all models trained)')

Comparing the most important features selected by the model with the most important features in Univariate Strategy, we can see that the Relative Strength Index calculated on 9 days and the Price to Earnings Ratio are present in both groups. The machine learning model, furthermore, selects as important features the Volatily computed on different periods, the Price to Tangible Book Value per Share and the Current Market Capitalization, that did not outperform in the Univariate Strategy.

This shows that the model can find different patterns than the simple univariate strategy, that allow to predict the future return.

In [ ]:
df_ret = res.drop('2003-12-31', level=0, axis=0)['RETURNS']
df_pred = res.drop('2003-12-31', level=0, axis=0)['PREDICTIONS']
print('Root Mean Squared Error (RMSE):', round(metrics.mean_squared_error(df_ret, df_pred, squared=False),ndigits=3))
print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(df_ret, df_pred),ndigits=3))
print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(df_ret, df_pred),ndigits=3))

The output metrics are good, showing a good ability of the model to predict the returns.

### Linear Regression

We apply a Linear Regression using the same method described before, to have a benchmark for the results obtained by the Random Forest Regressor.

In [ ]:
def xs_lr(df, n_estimators=4):
    '''Build a cross-sectional linear regression model and
    output the pipeline at each timestamp'''
    # sklearn does not have an imputing method unless specified
    df = df[df['RETURNS'].notna()]
    # do not leave out test sample in our training pipeline
    X_train, y_train = df.shift(1).drop(columns='RETURNS').values, df['RETURNS'].values

    model = make_pipeline(KNNImputer(n_neighbors=3),
                          LinearRegression())
    model.fit(X_train, y_train)
    # we return the model instead of slope estimations
    # such that we can transform the next-period data later
    return model

In [ ]:
ls_lr = {}
ts = df.index.to_list()
dt = 12                                # 1-year rolling window
for i in range(len(ts)-dt+1):
    df3 = data_stack.loc[ts[i]:ts[i+dt-1]]
    ls_lr[ts[i+dt-1]] = xs_lr(df3)    # add the model corresponding to i+dt-1, last period of rolling window
df_lr = pd.Series(ls_lr)

res_lr = {}

for t in df_lr.shift(1).index.to_list():
    df4 = data_stack.loc[t]
    df4 = df4[df4['RETURNS'].notna()]
    X_test, y_test = df4.shift(1).drop(columns='RETURNS').values, df4['RETURNS'].values
    try:
        lr = df_lr.shift(1).loc[t]
        y_pred = lr.predict(X_test).reshape(-1)  # prediction array reshaped to 1d
    except:
        y_pred = np.nan
    res_lr[t] = pd.DataFrame({'RETURNS': y_test, 'PREDICTIONS': y_pred}, index=df4.index)

res_lr = pd.concat(res_lr)
res_lr.index.names = ['Date', 'ticker']

In [ ]:
feature_importance_lr = pd.DataFrame()
for i in range(len(ts)-dt+1):
  feature_importance_lr[ts[i+dt-1]] = abs(ls_lr[ts[i+dt-1]][1].coef_)
impo_means_lr = feature_importance_lr.mean(axis = 1)
features_lr = pd.Series(impo_means_lr.values, data_stack.drop('RETURNS', axis=1).columns)
features_lr.sort_values(ascending=False).plot.bar(fontsize=7, title='Feature Importance expressed as absolute value of coefficient (average on all models trained)')

We can see that some important parameters selected by the Linear Regression are similar to the one selected by Univariate strategy: taking into consideration the seven most important features, both groups include Relative Strenght Index combuted at 9, 14 or 30 days, and Consesus Equity Recommendations. The Linear Regression, furthermore, selects WACC - Cost Equity and Normalized Accruals.

In [ ]:
df_ret_lr = res_lr.drop('2003-12-31', level=0, axis=0)['RETURNS']
df_pred_lr = res_lr.drop('2003-12-31', level=0, axis=0)['PREDICTIONS']

table = [['Root Mean Squared Error (RMSE):',
          round(metrics.mean_squared_error(df_ret_lr, df_pred_lr, squared=False),ndigits=3),
          round(metrics.mean_squared_error(df_ret, df_pred, squared=False),ndigits=3)
          ],
         ['Mean Absolute Error (MAE):',
          round(metrics.mean_absolute_error(df_ret_lr, df_pred_lr),ndigits=3),
          round(metrics.mean_absolute_error(df_ret, df_pred),ndigits=3)
          ],
         ['Mean Squared Error (MSE):',
          round(metrics.mean_squared_error(df_ret_lr, df_pred_lr),ndigits=3),
          round(metrics.mean_squared_error(df_ret, df_pred),ndigits=3)
          ]]

print(tabulate(table, headers=['','LR', 'RF']))

Comparing the metrics with the ones obtained with the Random Forest Regression, we can see that the ability of the Random Forest to predict the earning is way higher: its errors are, in fact, lower. Since Linear Regression is a very simple model, we expected this behaviour.
